线性回归的简洁实现

In [1]:
import numpy as np
import torch
from torch.utils import data  # 有一些处理数据的模块
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)  # 人工数据合成函数

调用框架中现有的API来读取数据

In [2]:
def load_array(data_arrays, batch_size, is_train=True):
    # 构造一个Pytorch数据迭代器
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)  # 每次随机挑选batch_size个样本出来，shuffle是看是否随机打乱


batch_size = 10
data_iter = load_array((features, labels), batch_size)  # 数据量太大的话load data要考虑内存占用率的问题

next(iter(data_iter))

[tensor([[ 0.6526, -1.1515],
         [ 1.0541,  0.2293],
         [-0.9705, -0.5926],
         [-2.1367,  0.5951],
         [-0.0494,  1.7532],
         [ 0.2263, -1.1444],
         [ 0.7915,  1.0854],
         [-1.1328, -1.6037],
         [-0.6059,  0.9304],
         [ 2.1690,  1.7730]]),
 tensor([[ 9.4121],
         [ 5.5361],
         [ 4.2775],
         [-2.0878],
         [-1.8522],
         [ 8.5488],
         [ 2.1074],
         [ 7.3971],
         [-0.1621],
         [ 2.5024]])]

使用框架预定义好的层

In [3]:
from torch import nn  # nn是神经网络的缩写

# nn.Sequential是一个有序的容器，神经网络模块将按照在传入构造器的顺序一次被添加到计算图中执行
net = nn.Sequential(nn.Linear(2, 1))  # 此处线性层输入的维度是2.输出的维度是1

初始化模型参数

In [4]:
net[0].weight.data.normal_(0, 0.01)  # normal_就是使用正态分布（均值为0，标准差为0.01）替换data的值
net[0].bias.data.fill_(0)  # 偏差直接设为0

tensor([0.])

计算均方误差使用的是MSELoss类，也称为平方${l_2}$范数

In [5]:
loss = nn.MSELoss()

实例化SGD实例

In [6]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)  # 至少传入两个参数，第一个参数是模型中的所有参数

训练过程代码

In [7]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:  # 每次拿出mini batch
        l = loss(net(X), y)  # net自带模型参数
        trainer.zero_grad()  # 先进行梯度清零
        l.backward()  # pytorch已经做了sum，不需要自己再做
        trainer.step()  # 调用step函数进行参数更新
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000219
epoch 2, loss 0.000104
epoch 3, loss 0.000103
